In [9]:
from keys import ml_prod_servicekey as ml_key
from ml_interface import MLFoundationClient
from database_interface import DBClient
from image_interface import ImageDBClient

In [10]:
# init the clients
db_client = DBClient("dummy")
ml_client = MLFoundationClient(ml_key, offline = True)
im_client = ImageDBClient()

In [13]:
img_path = "test-photo2.jpeg"
img = open(img_path, "rb")

In [14]:
# use this key to upload to the database
key = im_client.upload(img_path)

'cb144563-73f2-417f-a600-eced70ef7438'

In [7]:
img_defect_list = ml_client.modelPredictAll(img)